In [185]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from data_processing import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

warnings.simplefilter("ignore")
%matplotlib inline
%load_ext autoreload
%autoreload 2

sns.set(style="darkgrid")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [186]:
data = pd.read_csv("./train.csv")
data = data.drop(columns=["Id"])

y = data["SalePrice"]
X = data.drop(columns=["SalePrice"])

print(X.shape)
X.head()

(1460, 79)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [187]:
most_missing = get_missing_values_info(data)
most_missing

,Total,%
PoolQC,1453,99.520548
MiscFeature,1406,96.301370
Alley,1369,93.767123
Fence,1179,80.753425
FireplaceQu,690,47.260274
LotFrontage,259,17.739726
GarageYrBlt,81,5.547945
GarageCond,81,5.547945
GarageType,81,5.547945
GarageFinish,81,5.547945


In [188]:
data=data.drop(columns=['PoolQC', 'MiscFeature','Alley', 'Fence'], axis=1)

In [189]:
X = one_hot_encode(X)
X.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,1,0,0,0,0,1,0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,1,0,0,0,0,1,0
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,1,1,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,1,0,0,0,0,1,0


In [190]:
#magical seed
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=98987)

In [191]:
my_imputer = SimpleImputer()
X_train = my_imputer.fit_transform(X_train)
X_test = my_imputer.transform(X_test)

In [192]:
scaler = MinMaxScaler()

scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [193]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(np.log1p(y), np.log1p(y_pred)))

rmsle_score = make_scorer(rmsle, greater_is_better=False)

In [194]:
def evaluate(clf): 
    predictions = clf.predict(X_test_scaled)
    print("Mean Absolute log Error : " + str(rmsle(predictions, y_test)))

In [195]:
model = xgb.XGBRegressor(n_estimators = 1000)

model.fit(X_train_scaled, y_train, early_stopping_rounds=5, eval_set=[(X_test_scaled, y_test)], verbose=False)

evaluate(model)

Mean Absolute log Error : 0.16348001945424032


In [197]:
model = xgb.XGBRegressor(colsample_bytree=1.,
                 eta=0.01,
                 max_depth=4,
                 min_child_weight=1.5,
                 n_estimators=14400,                                                                  
                 alpha=0.,
                 reg_lambda=0.4,
                 subsample=0.2)

model.fit(X_train_scaled, y_train, early_stopping_rounds=15, eval_set=[(X_test_scaled, y_test)], verbose=False)

evaluate(model)

Mean Absolute log Error : 0.13971934454008733
